In [1]:
import os
import time
import pandas as pd
from nba_api.stats.static import players
from nba_api.stats.endpoints import playergamelog


In [2]:
# Create directory for player logs
os.makedirs("player_logs", exist_ok=True)

# Step 1: Get all known players
player_dict = players.get_players()

missed_df = pd.read_csv(r'C:\Users\ninuy\DSC 672\Data\Injury Database - Oct 2021 - June 2024.csv')
# Convert "Last, First" -> "First Last"
def format_name(name):
    if ',' in name:
        last, first = name.split(',', 1)
        return first.strip() + ' ' + last.strip()
    return name.strip()

missed_df['PLAYER'] = missed_df['PLAYER'].apply(format_name)

injured_names = set(missed_df['PLAYER'])

# Filter the dictionary
filtered_players = [player for player in player_dict if player["full_name"] in injured_names]



In [3]:
# Split filtered_players into 4 batches
batch_size = len(filtered_players) // 4
batches = [filtered_players[i * batch_size:(i + 1) * batch_size] for i in range(3)]
batches.append(filtered_players[3 * batch_size:])  # Last batch includes remainder

# Save each batch to separate CSVs (optional)
for i, batch in enumerate(batches):
    df = pd.DataFrame(batch)
    df.to_csv(f"filtered_players_batch_{i+1}.csv", index=False)
    print(f"💾 Saved batch {i+1} with {len(batch)} players")


💾 Saved batch 1 with 188 players
💾 Saved batch 2 with 188 players
💾 Saved batch 3 with 188 players
💾 Saved batch 4 with 190 players


In [4]:

# Load one of the 4 batches
batch_index = 3  # Change this to 1, 2, or 3 for other batches
filtered_players = pd.read_csv(f"filtered_players_batch_{batch_index+1}.csv").to_dict(orient='records')

# Step 2: Identify players active since 2016
seasons_of_interest = [f"2{year}" for year in range(16, 25)]  # '216' to '224'
active_since_2016 = []

print("🔍 Identifying players active since 2016...")

for player in filtered_players:
    player_id = str(player['id'])
    player_name = player['full_name']

    try:
        gamelog = playergamelog.PlayerGameLog(player_id=player_id, season='2016-17')
        df = gamelog.get_data_frames()[0]
        if not df.empty:
            active_since_2016.append(player)
            print(f"✅ Found: {player_name}")
    except Exception as e:
        print(f"⏭️ Skipping {player_name}: {e}")

    time.sleep(1.5)  # Respect API rate limits

print(f"✅ Total players active since 2016: {len(active_since_2016)}")

# Step 3: Collect full game logs for all relevant seasons
all_players_df = pd.DataFrame()

for player in filtered_players:
    player_id = str(player['id'])
    player_name = player['full_name']
    filename = f"player_logs/{player_id}_{player_name.replace(' ', '_')}.csv"

    try:
        gamelog = playergamelog.PlayerGameLog(player_id=player_id, season='ALL')
        df = gamelog.get_data_frames()[0]

        df = df[df['SEASON_ID'].isin(seasons_of_interest)]
        df['PLAYER_NAME'] = player_name
        df.to_csv(filename, index=False)
        print(f"📥 Saved log: {player_name}")
        time.sleep(2)  # Again, slow down requests
    except Exception as e:
        print(f"❌ Error with {player_name}: {e}")
        continue

    all_players_df = pd.concat([all_players_df, df], ignore_index=True)

# Step 4: Save combined dataset
batch_log_path = f"C:/Users/ninuy/DSC 672/data/nba_player_gamelogs_batch_{batch_index+1}.csv"
all_players_df.to_csv(batch_log_path, index=False)

print(f"🎉 All data collection complete. Total rows: {len(all_players_df)}")


🔍 Identifying players active since 2016...
✅ Found: Josh Richardson
✅ Found: Austin Rivers
✅ Found: Rajon Rondo
✅ Found: Derrick Rose
✅ Found: Terrence Ross
✅ Found: Terry Rozier
✅ Found: Ricky Rubio
✅ Found: D'Angelo Russell
✅ Found: Domantas Sabonis
✅ Found: Tomas Satoransky
✅ Found: Wayne Selden
✅ Found: Pascal Siakam
✅ Found: Marcus Smart
✅ Found: Ish Smith
✅ Found: Tony Snell
✅ Found: Nik Stauskas
✅ Found: Lance Stephenson
✅ Found: Garrett Temple
✅ Found: Isaiah Thomas
✅ Found: Klay Thompson
✅ Found: Tristan Thompson
✅ Found: Karl-Anthony Towns
✅ Found: P.J. Tucker
✅ Found: Myles Turner
✅ Found: Denzel Valentine
✅ Found: Fred VanVleet
✅ Found: Kemba Walker
✅ Found: John Wall
✅ Found: T.J. Warren
✅ Found: Russell Westbrook
✅ Found: Hassan Whiteside
✅ Found: Andrew Wiggins
✅ Found: Lou Williams
✅ Found: Justise Winslow
✅ Found: Christian Wood
✅ Found: Delon Wright
✅ Found: Thaddeus Young
✅ Found: Cody Zeller
✅ Found: Ivica Zubac
✅ Total players active since 2016: 39
📥 Saved log: Aus

In [5]:
batch_log_path = f"C:/Users/ninuy/DSC 672/data/nba_player_gamelogs_batch_{batch_index+1}.csv"
all_players_df.to_csv(batch_log_path, index=False)
print(f"✅ Saved batch {batch_index+1} logs: {len(all_players_df)} rows")

✅ Saved batch 4 logs: 0 rows


In [3]:
from nba_api.stats.library.parameters import SeasonAll
SeasonAll.all

'ALL'

In [1]:
from transformers import pipeline

# Test it
pipe = pipeline("sentiment-analysis")
print(pipe("This is great!"))


ModuleNotFoundError: Could not import module 'pipeline'. Are this object's requirements defined correctly?